In [134]:
import pandas as pd
import numpy as np

In [135]:
from io import StringIO

# Data for State Abbreviations and FIPS Codes
state_data = """
Alabama,AL,01
Alaska,AK,02
Arizona,AZ,04
Arkansas,AR,05
California,CA,06
Colorado,CO,08
Connecticut,CT,09
Delaware,DE,10
District of Columbia,DC,11
Florida,FL,12
Georgia,GA,13
Hawaii,HI,15
Idaho,ID,16
Illinois,IL,17
Indiana,IN,18
Iowa,IA,19
Kansas,KS,20
Kentucky,KY,21
Louisiana,LA,22
Maine,ME,23
Maryland,MD,24
Massachusetts,MA,25
Michigan,MI,26
Minnesota,MN,27
Mississippi,MS,28
Missouri,MO,29
Montana,MT,30
Nebraska,NE,31
Nevada,NV,32
New Hampshire,NH,33
New Jersey,NJ,34
New Mexico,NM,35
New York,NY,36
North Carolina,NC,37
North Dakota,ND,38
Ohio,OH,39
Oklahoma,OK,40
Oregon,OR,41
Pennsylvania,PA,42
Puerto Rico,PR,72
Rhode Island,RI,44
South Carolina,SC,45
South Dakota,SD,46
Tennessee,TN,47
Texas,TX,48
Utah,UT,49
Vermont,VT,50
Virginia,VA,51
Virgin Islands,VI,78
Washington,WA,53
West Virginia,WV,54
Wisconsin,WI,55
Wyoming,WY,56
"""

# Create DataFrame for state data
state_df = pd.read_csv(StringIO(state_data), header=None, names=['State', 'Postal', 'STATEFP'])

# Display the resulting DataFrame
print(state_df)


                   State Postal  STATEFP
0                Alabama     AL        1
1                 Alaska     AK        2
2                Arizona     AZ        4
3               Arkansas     AR        5
4             California     CA        6
5               Colorado     CO        8
6            Connecticut     CT        9
7               Delaware     DE       10
8   District of Columbia     DC       11
9                Florida     FL       12
10               Georgia     GA       13
11                Hawaii     HI       15
12                 Idaho     ID       16
13              Illinois     IL       17
14               Indiana     IN       18
15                  Iowa     IA       19
16                Kansas     KS       20
17              Kentucky     KY       21
18             Louisiana     LA       22
19                 Maine     ME       23
20              Maryland     MD       24
21         Massachusetts     MA       25
22              Michigan     MI       26
23             M

In [70]:
!pip install geopandas

In [148]:
import geopandas as gpd

# Load TIGER/Line Shapefile for counties (replace 'tl_2020_us_county.zip' with the appropriate file)
counties_shapefile = gpd.read_file('tl_2022_us_county.zip')

# Extract relevant information
counties_info = counties_shapefile[['GEOID', 'NAME', 'STATEFP', 'geometry']]

# Function to calculate latitude and longitude ranges for each county
def calculate_lat_lon_ranges(geometry):
    min_longitude, min_latitude, max_longitude, max_latitude = geometry.bounds
    return {
        'min_latitude': min_latitude,
        'max_latitude': max_latitude,
        'min_longitude': min_longitude,
        'max_longitude': max_longitude,
    }


# Apply the function to calculate ranges for each county
counties_info['lat_lon_ranges'] = counties_info['geometry'].apply(calculate_lat_lon_ranges)

# Display the resulting DataFrame with latitude and longitude ranges
print(counties_info[['GEOID', 'NAME', 'lat_lon_ranges']])

counties_info = counties_info[['GEOID', 'NAME', 'STATEFP', 'lat_lon_ranges']]
print(counties_info)

      GEOID       NAME                                     lat_lon_ranges
0     31039     Cuming  {'min_latitude': 41.742015, 'max_latitude': 42...
1     53069  Wahkiakum  {'min_latitude': 46.144532, 'max_latitude': 46...
2     35011    De Baca  {'min_latitude': 33.995413, 'max_latitude': 34...
3     31109  Lancaster  {'min_latitude': 40.523013, 'max_latitude': 41...
4     31129   Nuckolls  {'min_latitude': 40.002023, 'max_latitude': 40...
...     ...        ...                                                ...
3230  13123     Gilmer  {'min_latitude': 34.548495, 'max_latitude': 34...
3231  27135     Roseau  {'min_latitude': 48.538533, 'max_latitude': 49...
3232  28089    Madison  {'min_latitude': 32.399411, 'max_latitude': 32...
3233  48227     Howard  {'min_latitude': 32.087097, 'max_latitude': 32...
3234  54099      Wayne  {'min_latitude': 37.84242, 'max_latitude': 38....

[3235 rows x 3 columns]
      GEOID       NAME STATEFP  \
0     31039     Cuming      31   
1     53069  Wahkia

C:\Users\mihir\anaconda3\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [150]:
# Convert 'STATEFP' to string in counties_info DataFrame
counties_info['STATEFP'] = counties_info['STATEFP'].astype(int)
state_df['STATEFP'] = state_df['STATEFP'].astype(int)

# Merge DataFrames based on 'STATEFP'
merged_df = pd.merge(counties_info, state_df, on='STATEFP', how='outer', indicator=True)
merged_df = merged_df[['GEOID', 'NAME', 'STATEFP', 'State', 'lat_lon_ranges']]

print(merged_df)


      GEOID                       NAME  STATEFP        State  \
0     31039                     Cuming       31     Nebraska   
1     31109                  Lancaster       31     Nebraska   
2     31129                   Nuckolls       31     Nebraska   
3     31101                      Keith       31     Nebraska   
4     31137                     Phelps       31     Nebraska   
...     ...                        ...      ...          ...   
3230  09150   Northeastern Connecticut        9  Connecticut   
3231  09160            Northwest Hills        9  Connecticut   
3232  09170  South Central Connecticut        9  Connecticut   
3233  09180   Southeastern Connecticut        9  Connecticut   
3234  09190        Western Connecticut        9  Connecticut   

                                         lat_lon_ranges  
0     {'min_latitude': 41.742015, 'max_latitude': 42...  
1     {'min_latitude': 40.523013, 'max_latitude': 41...  
2     {'min_latitude': 40.002023, 'max_latitude': 40...  

In [151]:
print(merged_df[merged_df["NAME"] == "Duval"].to_string())

      GEOID   NAME  STATEFP    State                                                                                                    lat_lon_ranges
376   48131  Duval       48    Texas  {'min_latitude': 27.262487, 'max_latitude': 28.057995, 'min_longitude': -98.803325, 'max_longitude': -98.232464}
2567  12031  Duval       12  Florida  {'min_latitude': 30.103748, 'max_latitude': 30.586232, 'min_longitude': -82.049502, 'max_longitude': -81.316712}


In [152]:
df = pd.read_csv("Cleaned_AccidentsData_2010_2021.csv")
df

C:\Users\mihir\AppData\Local\Temp\ipykernel_15548\4146146591.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Cleaned_AccidentsData_2010_2021.csv")


,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,harm_evname,typ_intname,lgt_cond,lgt_condname,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year
0,2/23/2020 2:21:00 AM,Texas,480391,2020,0.0,1,3.0,BEXAR,SAN ANTONIO,Sunday,...,Impact Attenuator/Crash Cushion,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-480391
1,2/9/2020 10:50:00 PM,California,60083,2020,0.0,3,4.0,SAN BERNARDINO,Highlands,Sunday,...,Motor Vehicle In-Transport,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-60083
2,2/23/2020 8:57:00 PM,California,60510,2020,1.0,1,2.0,LOS ANGELES,Los Ángeles,Sunday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-60510
3,3/1/2020 1:15:00 AM,Missouri,290113,2020,0.0,1,1.0,JEFFERSON,ARNOLD,Sunday,...,Guardrail End,Not an Intersection,3,Dark - Lighted,1,Clear,1,26,1,2020-290113
4,3/15/2020 8:01:00 PM,Florida,120555,2020,0.0,1,1.0,BROWARD,DAVIE,Sunday,...,Concrete Traffic Barrier,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-120555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399719,4/4/2014 11:35:00 PM,Georgia,130236,2014,1.0,1,1.0,COBB,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-130236
399720,5/2/2014 8:44:00 PM,California,60692,2014,1.0,1,1.0,LOS ANGELES,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-60692
399721,5/16/2014 11:15:00 PM,Kentucky,210191,2014,1.0,1,2.0,KENTON,Erlangen,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,8,34,1,2014-210191
399722,7/25/2014 11:20:00 PM,Louisiana,220339,2014,0.0,1,1.0,WEST BATON ROUGE,Port Ellen,Friday,...,Curb,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-220339


In [158]:
df['county-name'] = df['county-name'].str.title()
df['statename'] = df['statename'].str.title()
merged_df['NAME'] = merged_df['NAME'].str.title()
merged_df['State'] = merged_df['State'].str.title()

In [159]:
# Filter rows in df based on latitude and longitude ranges
filtered_df = df.merge(
    merged_df,
    how='left',
    left_on=['statename','county-name'],
    right_on=['State','NAME']
)

# Apply filtering conditions
filtered_df['in_range'] = (
    (filtered_df['latitude'] >= filtered_df['lat_lon_ranges'].apply(lambda x: x.get('min_latitude') if isinstance(x, dict) else np.nan)) &
    (filtered_df['latitude'] <= filtered_df['lat_lon_ranges'].apply(lambda x: x.get('max_latitude') if isinstance(x, dict) else np.nan)) &
    (filtered_df['longitude'] >= filtered_df['lat_lon_ranges'].apply(lambda x: x.get('min_longitude') if isinstance(x, dict) else np.nan)) &
    (filtered_df['longitude'] <= filtered_df['lat_lon_ranges'].apply(lambda x: x.get('max_longitude') if isinstance(x, dict) else np.nan))
)

# Keep only rows within the specified ranges
filtered_df = filtered_df[filtered_df['in_range']]

# Drop unnecessary columns
filtered_df = filtered_df.drop(columns=['lat_lon_ranges', 'in_range'])

# Display the filtered DataFrame
print(filtered_df)


                         Date   statename  st_case  caseyear  pernotmvit  \
0        2/23/2020 2:21:00 AM       Texas   480391      2020         0.0   
1        2/9/2020 10:50:00 PM  California    60083      2020         0.0   
2        2/23/2020 8:57:00 PM  California    60510      2020         1.0   
3         3/1/2020 1:15:00 AM    Missouri   290113      2020         0.0   
4        3/15/2020 8:01:00 PM     Florida   120555      2020         0.0   
...                       ...         ...      ...       ...         ...   
403284   4/4/2014 11:35:00 PM     Georgia   130236      2014         1.0   
403285    5/2/2014 8:44:00 PM  California    60692      2014         1.0   
403286  5/16/2014 11:15:00 PM    Kentucky   210191      2014         1.0   
403287  7/25/2014 11:20:00 PM   Louisiana   220339      2014         0.0   
403288   8/15/2014 3:53:00 AM   Louisiana   220449      2014         1.0   

        ve_total  permvit       county-name        cityname day_weekname  ...  \
0     

In [162]:
filtered_df[(filtered_df["county-name"] == "Duval") & (filtered_df["statename"] == "Florida")]

,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year,GEOID,NAME,STATEFP,State
559,10/19/2020 6:05:00 PM,Florida,122579,2020,0.0,2,5.0,Duval,JACKSONVILLE,Monday,...,2,Rain,99,99,3,2020-122579,12031,Duval,12.0,Florida
678,11/3/2020 9:02:00 AM,Florida,122608,2020,0.0,2,2.0,Duval,JACKSONVILLE,Tuesday,...,1,Clear,99,99,1,2020-122608,12031,Duval,12.0,Florida
773,8/3/2020 3:22:00 PM,Florida,121779,2020,0.0,2,2.0,Duval,JACKSONVILLE,Monday,...,2,Rain,99,99,1,2020-121779,12031,Duval,12.0,Florida
890,2/24/2020 12:14:00 AM,Florida,120325,2020,0.0,2,2.0,Duval,NOT APPLICABLE,Monday,...,1,Clear,99,99,1,2020-120325,12031,Duval,12.0,Florida
1091,3/8/2020 3:28:00 AM,Florida,120388,2020,1.0,1,1.0,Duval,JACKSONVILLE,Sunday,...,1,Clear,99,99,1,2020-120388,12031,Duval,12.0,Florida
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402003,4/4/2014 5:25:00 PM,Florida,120390,2014,0.0,4,16.0,Duval,JACKSONVILLE,Friday,...,1,Clear,99,99,1,2014-120390,12031,Duval,12.0,Florida
402051,8/8/2014 4:05:00 PM,Florida,121315,2014,0.0,2,2.0,Duval,JACKSONVILLE,Friday,...,2,Rain,99,99,1,2014-121315,12031,Duval,12.0,Florida
402345,6/6/2014 3:21:00 PM,Florida,120821,2014,2.0,1,1.0,Duval,JACKSONVILLE,Friday,...,1,Clear,99,99,1,2014-120821,12031,Duval,12.0,Florida
402439,4/11/2014 3:00:00 PM,Florida,120634,2014,0.0,4,10.0,Duval,JACKSONVILLE,Friday,...,10,Cloudy,99,99,2,2014-120634,12031,Duval,12.0,Florida


In [189]:
filtered_df = filtered_df.drop(columns=["GEOID","NAME","STATEFP","State"])

In [190]:
filtered_df

,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,harm_evname,typ_intname,lgt_cond,lgt_condname,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year
0,2/23/2020 2:21:00 AM,Texas,480391,2020,0.0,1,3.0,Bexar,SAN ANTONIO,Sunday,...,Impact Attenuator/Crash Cushion,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-480391
1,2/9/2020 10:50:00 PM,California,60083,2020,0.0,3,4.0,San Bernardino,Highlands,Sunday,...,Motor Vehicle In-Transport,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-60083
2,2/23/2020 8:57:00 PM,California,60510,2020,1.0,1,2.0,Los Angeles,Los Ángeles,Sunday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-60510
3,3/1/2020 1:15:00 AM,Missouri,290113,2020,0.0,1,1.0,Jefferson,ARNOLD,Sunday,...,Guardrail End,Not an Intersection,3,Dark - Lighted,1,Clear,1,26,1,2020-290113
4,3/15/2020 8:01:00 PM,Florida,120555,2020,0.0,1,1.0,Broward,DAVIE,Sunday,...,Concrete Traffic Barrier,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-120555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403284,4/4/2014 11:35:00 PM,Georgia,130236,2014,1.0,1,1.0,Cobb,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-130236
403285,5/2/2014 8:44:00 PM,California,60692,2014,1.0,1,1.0,Los Angeles,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-60692
403286,5/16/2014 11:15:00 PM,Kentucky,210191,2014,1.0,1,2.0,Kenton,Erlangen,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,8,34,1,2014-210191
403287,7/25/2014 11:20:00 PM,Louisiana,220339,2014,0.0,1,1.0,West Baton Rouge,Port Ellen,Friday,...,Curb,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-220339


In [248]:
filtered_df["lgt_condname"].value_counts()

lgt_condname
Daylight                   184681
Dark - Not Lighted         108125
Dark - Lighted              75534
Dusk                         9098
Dawn                         7312
Dark - Unknown Lighting      2703
Unknown                       798
Reported as Unknown           550
Not Reported                  374
Other                         169
Name: count, dtype: int64

In [250]:
# Assuming 'filtered_df' is your DataFrame and 'lgt_condname' is the column of interest
value_counts = filtered_df["lgt_condname"].value_counts()

# Identify values with counts under 1000
values_to_merge = value_counts[value_counts < 1000].index

# Replace those values with 'others'
filtered_df.loc[filtered_df["lgt_condname"].isin(values_to_merge), "lgt_condname"] = 'others'

# Check the modified DataFrame
filtered_df["lgt_condname"].value_counts()


lgt_condname
Daylight                   184681
Dark - Not Lighted         108125
Dark - Lighted              75534
Dusk                         9098
Dawn                         7312
Dark - Unknown Lighting      2703
others                       1891
Name: count, dtype: int64

In [251]:
filtered_df

,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,harm_evname,typ_intname,lgt_cond,lgt_condname,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year
0,2/23/2020 2:21:00 AM,Texas,480391,2020,0.0,1,3.0,Bexar,SAN ANTONIO,Sunday,...,Impact Attenuator/Crash Cushion,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-480391
1,2/9/2020 10:50:00 PM,California,60083,2020,0.0,3,4.0,San Bernardino,Highlands,Sunday,...,Motor Vehicle In-Transport,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-60083
2,2/23/2020 8:57:00 PM,California,60510,2020,1.0,1,2.0,Los Angeles,Los Ángeles,Sunday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-60510
3,3/1/2020 1:15:00 AM,Missouri,290113,2020,0.0,1,1.0,Jefferson,ARNOLD,Sunday,...,Guardrail End,Not an Intersection,3,Dark - Lighted,1,Clear,1,26,1,2020-290113
4,3/15/2020 8:01:00 PM,Florida,120555,2020,0.0,1,1.0,Broward,DAVIE,Sunday,...,Concrete Traffic Barrier,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-120555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403284,4/4/2014 11:35:00 PM,Georgia,130236,2014,1.0,1,1.0,Cobb,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-130236
403285,5/2/2014 8:44:00 PM,California,60692,2014,1.0,1,1.0,Los Angeles,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-60692
403286,5/16/2014 11:15:00 PM,Kentucky,210191,2014,1.0,1,2.0,Kenton,Erlangen,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,8,34,1,2014-210191
403287,7/25/2014 11:20:00 PM,Louisiana,220339,2014,0.0,1,1.0,West Baton Rouge,Port Ellen,Friday,...,Curb,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-220339


In [252]:
filtered_df.to_csv("filteredData_MihirApproach.csv")

# ABHI's APPROACH

In [164]:
dfA = pd.read_csv("Cleaned_AccidentsData_2010_2021.csv")
dfA

C:\Users\mihir\AppData\Local\Temp\ipykernel_15548\1634842913.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  dfA = pd.read_csv("Cleaned_AccidentsData_2010_2021.csv")


,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,harm_evname,typ_intname,lgt_cond,lgt_condname,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year
0,2/23/2020 2:21:00 AM,Texas,480391,2020,0.0,1,3.0,BEXAR,SAN ANTONIO,Sunday,...,Impact Attenuator/Crash Cushion,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-480391
1,2/9/2020 10:50:00 PM,California,60083,2020,0.0,3,4.0,SAN BERNARDINO,Highlands,Sunday,...,Motor Vehicle In-Transport,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-60083
2,2/23/2020 8:57:00 PM,California,60510,2020,1.0,1,2.0,LOS ANGELES,Los Ángeles,Sunday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-60510
3,3/1/2020 1:15:00 AM,Missouri,290113,2020,0.0,1,1.0,JEFFERSON,ARNOLD,Sunday,...,Guardrail End,Not an Intersection,3,Dark - Lighted,1,Clear,1,26,1,2020-290113
4,3/15/2020 8:01:00 PM,Florida,120555,2020,0.0,1,1.0,BROWARD,DAVIE,Sunday,...,Concrete Traffic Barrier,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-120555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399719,4/4/2014 11:35:00 PM,Georgia,130236,2014,1.0,1,1.0,COBB,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-130236
399720,5/2/2014 8:44:00 PM,California,60692,2014,1.0,1,1.0,LOS ANGELES,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-60692
399721,5/16/2014 11:15:00 PM,Kentucky,210191,2014,1.0,1,2.0,KENTON,Erlangen,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,8,34,1,2014-210191
399722,7/25/2014 11:20:00 PM,Louisiana,220339,2014,0.0,1,1.0,WEST BATON ROUGE,Port Ellen,Friday,...,Curb,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-220339


In [168]:
dfA['State/County'] = dfA['statename'].astype(str) + '-' + dfA['county-name'].astype(str)
dfA

,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,typ_intname,lgt_cond,lgt_condname,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year,State/County
0,2/23/2020 2:21:00 AM,Texas,480391,2020,0.0,1,3.0,BEXAR,SAN ANTONIO,Sunday,...,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-480391,Texas-BEXAR
1,2/9/2020 10:50:00 PM,California,60083,2020,0.0,3,4.0,SAN BERNARDINO,Highlands,Sunday,...,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-60083,California-SAN BERNARDINO
2,2/23/2020 8:57:00 PM,California,60510,2020,1.0,1,2.0,LOS ANGELES,Los Ángeles,Sunday,...,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-60510,California-LOS ANGELES
3,3/1/2020 1:15:00 AM,Missouri,290113,2020,0.0,1,1.0,JEFFERSON,ARNOLD,Sunday,...,Not an Intersection,3,Dark - Lighted,1,Clear,1,26,1,2020-290113,Missouri-JEFFERSON
4,3/15/2020 8:01:00 PM,Florida,120555,2020,0.0,1,1.0,BROWARD,DAVIE,Sunday,...,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-120555,Florida-BROWARD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399719,4/4/2014 11:35:00 PM,Georgia,130236,2014,1.0,1,1.0,COBB,NOT APPLICABLE,Friday,...,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-130236,Georgia-COBB
399720,5/2/2014 8:44:00 PM,California,60692,2014,1.0,1,1.0,LOS ANGELES,NOT APPLICABLE,Friday,...,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-60692,California-LOS ANGELES
399721,5/16/2014 11:15:00 PM,Kentucky,210191,2014,1.0,1,2.0,KENTON,Erlangen,Friday,...,Not an Intersection,3,Dark - Lighted,1,Clear,8,34,1,2014-210191,Kentucky-KENTON
399722,7/25/2014 11:20:00 PM,Louisiana,220339,2014,0.0,1,1.0,WEST BATON ROUGE,Port Ellen,Friday,...,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-220339,Louisiana-WEST BATON ROUGE


In [192]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

def detect_outliers(df, eps, min_samples):
    X1 = df['latitude'].to_numpy().reshape(-1, 1)
    X2 = df['longitude'].to_numpy().reshape(-1, 1)

    dbscan1 = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan2 = DBSCAN(eps=eps, min_samples=min_samples)

    dbscan1.fit(X1)
    dbscan2.fit(X2)

    labels1 = dbscan1.labels_
    labels2 = dbscan2.labels_

    outliers1 = np.where(labels1 == -1)[0]
    outliers2 = np.where(labels2 == -1)[0]

    df["anomaly_score_latitude"] = labels1
    df["anomaly_score_longitude"] = labels2
    
#     print(df)
#     print("__________________________________________________________________________")
    return df

# Assuming your dataframe is named 'df'
eps = 0.1
min_samples = 5

# Apply outlier detection for each county using groupby
df_outliers = dfA.groupby('State/County').apply(lambda x: detect_outliers(x, eps, min_samples)).reset_index(drop=True)

# Concatenate the results
all_counties_df = pd.concat([df_outliers], ignore_index=True)

# Resetting index after concatenation
all_counties_df.reset_index(drop=True, inplace=True)


In [193]:
# Assuming 'all_counties_df' is your DataFrame
all_counties_df = all_counties_df[(all_counties_df["anomaly_score_longitude"] != -1) & (all_counties_df["anomaly_score_latitude"] != -1)]

# Resetting index after filtering
all_counties_df.reset_index(drop=True, inplace=True)
all_counties_df = all_counties_df.drop(columns=["anomaly_score_latitude", "anomaly_score_longitude", "State/County"])
all_counties_df

,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,harm_evname,typ_intname,lgt_cond,lgt_condname,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year
0,12/14/2020 4:50:00 AM,Alabama,10855,2020,0.0,1,1.0,AUTAUGA,NOT APPLICABLE,Monday,...,Live Animal,Not an Intersection,2,Dark - Not Lighted,10,Cloudy,5,5,1,2020-10855
1,7/31/2010 12:30:00 AM,Alabama,10400,2010,NaN,1,NaN,AUTAUGA,NOT APPLICABLE,Saturday,...,Tree (Standing Only),Not an Intersection,3,Dark - Lighted,1,Clear,1,0,1,2010-10400
2,8/6/2010 9:05:00 PM,Alabama,10406,2010,NaN,1,NaN,AUTAUGA,NOT APPLICABLE,Friday,...,Traffic Sign Support,Not an Intersection,3,Dark - Lighted,1,Clear,21,25,1,2010-10406
3,8/23/2010 6:29:00 AM,Alabama,10546,2010,NaN,2,NaN,AUTAUGA,Prattsville,Monday,...,Motor Vehicle In-Transport,Four-Way Intersection,1,Daylight,1,Clear,6,36,1,2010-10546
4,11/15/2020 6:06:00 PM,Alabama,10727,2020,0.0,2,2.0,AUTAUGA,Prattsville,Sunday,...,Motor Vehicle In-Transport,Not an Intersection,3,Dark - Lighted,1,Clear,19,7,1,2020-10727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394910,5/3/2016 6:40:00 PM,Wyoming,560015,2016,0.0,2,5.0,WESTON,NOT APPLICABLE,Tuesday,...,Motor Vehicle In-Transport,Not an Intersection,1,Daylight,1,Clear,18,52,1,2016-560015
394911,11/27/2017 8:04:00 AM,Wyoming,560100,2017,0.0,1,1.0,WESTON,NOT APPLICABLE,Monday,...,Embankment,Not an Intersection,1,Daylight,1,Clear,8,11,1,2017-560100
394912,8/12/2011 3:20:00 PM,Wyoming,560070,2011,0.0,1,2.0,WESTON,NOT APPLICABLE,Friday,...,Embankment,Not an Intersection,1,Daylight,1,Clear,15,35,1,2011-560070
394913,6/22/2011 3:25:00 PM,Wyoming,560041,2011,0.0,2,4.0,WESTON,NOT APPLICABLE,Wednesday,...,Motor Vehicle In-Transport,Not an Intersection,1,Daylight,1,Clear,15,38,2,2011-560041


In [239]:
filtered_df

,Date,statename,st_case,caseyear,pernotmvit,ve_total,permvit,county-name,cityname,day_weekname,...,harm_evname,typ_intname,lgt_cond,lgt_condname,weather,weathername,arr_hour,arr_min,fatals,st_case_per_year
0,2/23/2020 2:21:00 AM,Texas,480391,2020,0.0,1,3.0,Bexar,SAN ANTONIO,Sunday,...,Impact Attenuator/Crash Cushion,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-480391
1,2/9/2020 10:50:00 PM,California,60083,2020,0.0,3,4.0,San Bernardino,Highlands,Sunday,...,Motor Vehicle In-Transport,Not an Intersection,3,Dark - Lighted,10,Cloudy,99,99,1,2020-60083
2,2/23/2020 8:57:00 PM,California,60510,2020,1.0,1,2.0,Los Angeles,Los Ángeles,Sunday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-60510
3,3/1/2020 1:15:00 AM,Missouri,290113,2020,0.0,1,1.0,Jefferson,ARNOLD,Sunday,...,Guardrail End,Not an Intersection,3,Dark - Lighted,1,Clear,1,26,1,2020-290113
4,3/15/2020 8:01:00 PM,Florida,120555,2020,0.0,1,1.0,Broward,DAVIE,Sunday,...,Concrete Traffic Barrier,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2020-120555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403284,4/4/2014 11:35:00 PM,Georgia,130236,2014,1.0,1,1.0,Cobb,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-130236
403285,5/2/2014 8:44:00 PM,California,60692,2014,1.0,1,1.0,Los Angeles,NOT APPLICABLE,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-60692
403286,5/16/2014 11:15:00 PM,Kentucky,210191,2014,1.0,1,2.0,Kenton,Erlangen,Friday,...,Pedestrian,Not an Intersection,3,Dark - Lighted,1,Clear,8,34,1,2014-210191
403287,7/25/2014 11:20:00 PM,Louisiana,220339,2014,0.0,1,1.0,West Baton Rouge,Port Ellen,Friday,...,Curb,Not an Intersection,3,Dark - Lighted,1,Clear,99,99,1,2014-220339
